In [1]:
import pyarrow.parquet as pa
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import numpy as np
from sklearn.decomposition import PCA
import _1_Preprocessing


In [5]:
df_train_pre, df_test_pre = _1_Preprocessing.run_preprocessing()

df_train=df_train_pre.drop('service_time_end',axis=1)
df_train=df_train.drop(['trip_id', 'order_time', 'order_datetime'],axis=1)
df_test=df_test_pre.drop('service_time_end',axis=1)
df_test=df_test.drop(['trip_id', 'order_time', 'order_datetime'],axis=1)
df_test

X_train = df_train.drop('service_time_in_minutes', axis=1)
y_train = df_train['service_time_in_minutes']
y_train.round(2)
X_train


X_test = df_test.drop('service_time_in_minutes', axis=1)
y_test = df_test['service_time_in_minutes']
y_test.round(2)
X_test

# train_preprocess_x = PCA().fit_transform(train_preprocess_x)
# test_preprocess_x = PCA().fit_transform(test_preprocess_x)



size before remove outliers:  (1534689, 14)
size after remove outliers:  (1409359, 14)
size after add article total weight:  (1409359, 15)
size after add crate count:  (1409359, 16)
size after handle missing values:  (1284697, 16)
size after service time start ordinal encoding:  (1284697, 16)
size after train test split:  (1027758, 16) (256939, 16)


,warehouse_id,has_elevator,floor,is_pre_order,is_business,web_order_id,customer_id,service_time_start,driver_id,article_weight_in_g,crate_count,num_previous_orders_customer,customer_speed
3,9,False,0.0,False,True,1463699,193987,13,9591,111914.0,8.0,5.0,4.0
5,46,False,1.0,False,True,248057,193995,12,8148,117385.0,10.0,2.0,4.0
13,18,False,0.0,True,True,1042860,166858,10,4765,76350.0,5.0,2.0,4.0
18,26,True,6.0,True,True,591036,299448,9,724,267661.0,17.0,2.0,4.0
28,1,False,3.0,False,True,1443245,156800,13,4949,69000.0,4.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409340,23,False,1.0,False,True,1423019,516968,12,3230,196300.0,13.0,4.0,4.0
1409348,19,False,0.0,True,True,235038,514140,12,1289,98600.0,6.0,2.0,4.0
1409350,19,False,0.0,True,True,1344902,514140,8,10520,74500.0,5.0,2.0,4.0
1409354,4,False,0.0,True,True,460582,433427,17,3793,260501.0,20.0,2.0,4.0


# RandomForestRegressor

In [6]:
regressor = RandomForestRegressor(n_estimators=20)

regressor.fit(X_train, y_train)

predictions = regressor.predict(X_train)

mse = mean_squared_error(y_train, predictions)
print(f'Mean Squared Error train: {mse}')

mae = mean_absolute_error(y_train, predictions)
print(f'Mean Absolute Error train: {mae}')

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error test: {mse}')

mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error test: {mae}')

Mean Squared Error train: 3.830126682786456
Mean Absolute Error train: 1.3056323938684493
Mean Squared Error test: 24.685992906013333
Mean Absolute Error test: 3.4060762075154014


# XGBoost

In [15]:
# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Due to previous Hyperparameter optimization we will use i = 1.4 and depth=1.9

params = {"objective": "reg:squarederror", "max_depth": 6,"gamma": 1.8}

n = 20000
model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
)

## Train
preds = model.predict(dtrain_reg)

mse = mean_squared_error(y_train, preds)
mae = mean_absolute_error(y_train, preds)

print(f"Train MSE of the base model: {mse:.3f}")
print(f"Train MAE of the base model: {mae:.3f}")

#### Test

preds = model.predict(dtest_reg)

mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(f"Test MSE of the base model: {mse:.3f}")
print(f"Test MAE of the base model: {mae:.3f}")


Train MSE of the base model: 18.243
Train MAE of the base model: 2.905
Test MSE of the base model: 21.999
Test MAE of the base model: 3.127


# Confidence Evaluation

## XGBoost

In [13]:
n = len(preds)
mu = np.mean(preds)
sigma = np.std(preds, ddof=1)  # unbiased estimator (divides by n-1)
z = sigma / np.sqrt(n)

# For 95% CI, use Z-value 1.96 directly
z_value = 1.96

lower_bound = (z_value * z) - mu
upper_bound = (z_value * z) + mu

print(lower_bound, upper_bound)

-9.76410043472284 9.794891951141418


## Random Forest

In [14]:
n = len(predictions)
mu = np.mean(predictions)
sigma = np.std(predictions, ddof=1)  # unbiased estimator (divides by n-1)
z = sigma / np.sqrt(n)

# For 95% CI, use Z-value 1.96 directly
z_value = 1.96

lower_bound = (z_value * z) - mu
upper_bound = (z_value * z) + mu

print(lower_bound, upper_bound)

-9.93846564286259 9.969793165035002


In [17]:
# Calculate variance
variance = np.var(predictions)

# Calculate standard deviation
std_deviation = np.std(predictions)

print(f"Variance: {variance}")
print(f"Standard Deviation: {std_deviation}")

Variance: 16.409989277770357
Standard Deviation: 4.050924496676081
